# Exercise 2: Advanced Analytics NLP

In [1]:
!pip install spark-nlp==1.7.3

    100% |████████████████████████████████| 72.8MB 629kB/s eta 0:00:01 0% |                                | 122kB 4.5MB/s eta 0:00:17    2% |▊                               | 1.6MB 13.5MB/s eta 0:00:06    7% |██▎                             | 5.2MB 22.3MB/s eta 0:00:04    8% |██▊                             | 6.2MB 20.4MB/s eta 0:00:04    9% |███▏                            | 7.2MB 20.2MB/s eta 0:00:04    10% |███▍                            | 7.6MB 9.5MB/s eta 0:00:07    14% |████▊                           | 10.7MB 22.8MB/s eta 0:00:03    28% |█████████▏                      | 20.9MB 22.1MB/s eta 0:00:03    31% |██████████                      | 22.9MB 19.0MB/s eta 0:00:03    34% |███████████                     | 24.9MB 20.0MB/s eta 0:00:03    39% |████████████▋                   | 28.8MB 20.5MB/s eta 0:00:03    42% |█████████████▌                  | 30.8MB 20.3MB/s eta 0:00:03    43% |██████████████                  | 31.8MB 21.4MB/s eta 0:00:02    46% |██████████████▉            

In [2]:
import pandas as pd
pd.set_option('max_colwidth', 800)

# Create a spark context that includes a 3rd party jar for NLP

In [3]:
#jarPath = "spark-nlp-assembly-1.7.3.jar"

from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .config("spark.jars.packages", "JohnSnowLabs:spark-nlp:1.8.2") \
    .getOrCreate()
spark

# Read multiple files in a dir as one Dataframe

In [4]:
dataPath = "./data/reddit/*.json"
df = spark.read.json(dataPath)
print(df.count())
df.printSchema()

100
root
 |-- data: struct (nullable = true)
 |    |-- approved_at_utc: string (nullable = true)
 |    |-- approved_by: string (nullable = true)
 |    |-- archived: boolean (nullable = true)
 |    |-- author: string (nullable = true)
 |    |-- author_flair_background_color: string (nullable = true)
 |    |-- author_flair_css_class: string (nullable = true)
 |    |-- author_flair_richtext: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- author_flair_template_id: string (nullable = true)
 |    |-- author_flair_text: string (nullable = true)
 |    |-- author_flair_text_color: string (nullable = true)
 |    |-- author_flair_type: string (nullable = true)
 |    |-- author_fullname: string (nullable = true)
 |    |-- author_patreon_flair: boolean (nullable = true)
 |    |-- banned_at_utc: string (nullable = true)
 |    |-- banned_by: string (nullable = true)
 |    |-- can_gild: boolean (nullable = true)
 |    |-- can_mod_post: boolean (nullable = true)


# Deal with Struct type to query subfields 

In [6]:
title = "data.title"
author = "data.author"
dfAuthorTitle = df.select(title, author)
dfAuthorTitle.limit(5).toPandas()


,title,author
0,"Microsoft Corp said it has discovered hacking targeting democratic institutions, think tanks, and non-profit organizations in Europe.",jaykirsch
1,Deutsche Bank reportedly planned to extend the dates of $340 million in loans to Trump Organization to avoid a potential nightmare of chasing a sitting president for cash,canuck_burger
2,"Iranian ""morality police"" were forced to fire warning shots when a crowd intervened to prevent them from arresting two women for not wearing a hijab. The incident occurred in Tehran's northeastern Narmak neighbourhood on Friday night, and ended with a mob tearing the door off a police vehicle.",honolulu_oahu_mod
3,"Trump administration 'pushing Saudi nuclear deal' which could benefit company linked to Jared Kushner - Senior Trump administration officials pushed a project to share nuclear power technology with Saudi Arabia over the objections of ethics officials, according to a congressional report",madam1
4,"NASA Happily Reports the Earth is Greener, With More Trees Than 20 Years Ago–and It's Thanks to China, India",purplexxx


# Try to implement the equivalent of flatMap in dataframes

In [ ]:
import pyspark.sql.functions as F



# Use an NLP libary to do Part-of-Speech Tagging

In [ ]:
from com.johnsnowlabs.nlp.pretrained.pipeline.en import BasicPipeline as bp
# Todo

## Deal with Map type to query subfields

In [ ]:
dfPos = dfAnnotated.select("text", "pos.metadata", "pos.result")
dfPos.limit(5).toPandas()

In [ ]:
dfPos= dfAnnotated.select(F.explode("pos").alias("pos"))
dfPos.printSchema()
dfPos.toPandas()

## Keep only proper nouns NNP or NNPS

In [ ]:
nnpFilter = "pos.result = 'NNP' or pos.result = 'NNPS' "
dfNNP = dfPos.where(nnpFilter)
dfNNP.limit(10).toPandas()

## Extract columns form a map in a col

In [ ]:
dfWordTag = dfNNP.selectExpr("pos.metadata['word'] as word", "pos.result as tag")
dfWordTag.limit(10).toPandas()

In [ ]:
from pyspark.sql.functions import desc
dfWordTag.groupBy("word").count().orderBy(desc("count")).show()